## TOPIC -Chest XRAY Image Classification in normal, covid, Tuberculosis and Pneumonia

## Importing libraries

In [1]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping

%matplotlib inline

## Loading the dataset

In [2]:
my_data_dir = 'C:\Users\Muleh\OneDrive\Desktop\om summer\CODE.ipynb'
test_path = my_data_dir+'/test/'
train_path = my_data_dir+'/train/'
val_path = my_data_dir+'/val/'

## Rescaling datset

In [15]:
import cv2
import os
path = 'C:\Users\Muleh\OneDrive\Desktop\om summer\CODE.ipynb/train/'
file_path = os.listdir(path)

for i in file_path:
    path_final = 'C:\Users\Muleh\OneDrive\Desktop\om summer\CODE.ipynb/train/' + i + '/'
    final = os.listdir(path_final)
    for j in final:
        data = path_final + j
        img = cv2.imread(data)
        image = cv2.resize(img, (224,224))
        cv2.imwrite(data,image)
#         print(data.shape)


In [16]:
import cv2
import os
path = 'C:\Users\Muleh\OneDrive\Desktop\om summer\CODE.ipynb/val/'
file_path = os.listdir(path)

for i in file_path:
    path_final = 'C:\Users\Muleh\OneDrive\Desktop\om summer\CODE.ipynb/val/' + i + '/'
    final = os.listdir(path_final)
    for j in final:
        data = path_final + j
        img = cv2.imread(data)
        image = cv2.resize(img, (224,224))
        cv2.imwrite(data,image)

In [17]:
import cv2
import os
path = 'C:\Users\Muleh\OneDrive\Desktop\om summer\CODE.ipynb/test/'
file_path = os.listdir(path)

for i in file_path:
    path_final = 'C:\Users\Muleh\OneDrive\Desktop\om summer\CODE.ipynb/test/' + i + '/'
    final = os.listdir(path_final)
    for j in final:
        data = path_final + j
        img = cv2.imread(data)
        image = cv2.resize(img, (224,224))
        cv2.imwrite(data,image)

## Now, we are using the image data generator function

In [18]:
image_shape = (224,224)
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.10,
                               height_shift_range=0.10,
                               rescale=1/255,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

## Building a convolutional neural network model

In [19]:
import tensorflow as tf

In [31]:

batch_size = 16

train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size= (224,224),
                                                
                                               batch_size=batch_size,
                                               class_mode='categorical')

Found 6326 images belonging to 4 classes.


In [32]:
val_image_gen = image_gen.flow_from_directory(val_path,
                                               target_size= (224,224),
                                              
                                               batch_size=batch_size,
                                               class_mode='categorical',shuffle=False)

Found 38 images belonging to 4 classes.


In [33]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size= (224,224),
                                              
                                               batch_size=batch_size,
                                               class_mode='categorical',shuffle=False)

Found 771 images belonging to 4 classes.


In [34]:
print(train_image_gen.class_indices)
Labels=train_image_gen.class_indices
print(val_image_gen.class_indices)

{'COVID19': 0, 'NORMAL': 1, 'PNEUMONIA': 2, 'TURBERCULOSIS': 3}
{'COVID19': 0, 'NORMAL': 1, 'PNEUMONIA': 2, 'TURBERCULOSIS': 3}


In [35]:
train_image_gen[1][0].shape

(16, 224, 224, 3)

## MobileNetV2

In [36]:

base_model = tf.keras.applications.MobileNetV2(input_shape = (224,224, 3), include_top = False, weights = "imagenet")

In [37]:
base_model.trainable = False

In [38]:
model = tf.keras.Sequential([base_model,
                                 tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(4096, activation="relu"),
                             tf.keras.layers.Dense(2048, activation="relu"),
                                 tf.keras.layers.Dense(1024, activation="relu"),
                             tf.keras.layers.Dense(512, activation="relu"),
                             tf.keras.layers.Dense(256, activation="relu"),
                             
                             tf.keras.layers.Dense(128, activation="relu"),
                                 tf.keras.layers.Dense(4, activation="softmax")                                     
                                ])

In [39]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_image_gen, validation_data = val_image_gen, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 58s 537ms/step - loss: 0.1984 - accuracy: 0.8419 - val_loss: 0.3735 - val_accuracy: 0.6579
Epoch 2/10
100/100 [==============================] - 51s 507ms/step - loss: 0.1133 - accuracy: 0.9162 - val_loss: 0.3021 - val_accuracy: 0.7368
Epoch 3/10
100/100 [==============================] - 48s 480ms/step - loss: 0.0876 - accuracy: 0.9350 - val_loss: 0.2013 - val_accuracy: 0.8158
Epoch 4/10
100/100 [==============================] - 50s 496ms/step - loss: 0.0872 - accuracy: 0.9344 - val_loss: 0.1881 - val_accuracy: 0.7895
Epoch 5/10
100/100 [==============================] - 49s 486ms/step - loss: 0.0881 - accuracy: 0.9308 - val_loss: 0.2617 - val_accuracy: 0.7368
Epoch 6/10
100/100 [==============================] - 49s 493ms/step - loss: 0.0659 - accuracy: 0.9463 - val_loss: 0.1622 - val_accuracy: 0.8684
Epoch 7/10
100/100 [==============================] - 49s 485ms/step - loss: 0.0689 - accuracy: 0.9538 - val_loss: 0.2641 - val_ac

In [40]:
predictions = np.argmax(model.predict(test_image_gen), axis = -1)
predictions

49/49 [==============================] - 25s 488ms/step


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 2, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 1, 2,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2,
       2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1,

In [41]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_image_gen.classes,predictions))

              precision    recall  f1-score   support

           0       1.00      0.88      0.93       106
           1       0.91      0.68      0.78       234
           2       0.82      0.97      0.89       390
           3       0.97      0.95      0.96        41

    accuracy                           0.87       771
   macro avg       0.93      0.87      0.89       771
weighted avg       0.88      0.87      0.87       771



In [42]:
print(confusion_matrix(test_image_gen.classes,predictions))

[[ 93   3   9   1]
 [  0 159  75   0]
 [  0  11 379   0]
 [  0   1   1  39]]


## InceptionV3

In [43]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [44]:
base_model = InceptionV3(input_shape = (224, 224, 3), include_top = False)

87910968/87910968 [==============================] - 10s 0us/step


In [45]:

for layer in base_model.layers:
    layer.trainable = False

In [48]:
from tensorflow.keras.optimizers import RMSprop

x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(2048, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = tf.keras.layers.Dense(4, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [49]:
inc_history = model.fit_generator(train_image_gen, validation_data = val_image_gen, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 204s 2s/step - loss: 0.5362 - acc: 0.7019 - val_loss: 0.3642 - val_acc: 0.6842
Epoch 2/10
100/100 [==============================] - 189s 2s/step - loss: 0.2118 - acc: 0.8333 - val_loss: 0.4230 - val_acc: 0.6316
Epoch 3/10
100/100 [==============================] - 187s 2s/step - loss: 0.1679 - acc: 0.8667 - val_loss: 0.3151 - val_acc: 0.7368
Epoch 4/10
100/100 [==============================] - 195s 2s/step - loss: 0.1680 - acc: 0.8650 - val_loss: 0.4791 - val_acc: 0.6842
Epoch 5/10
100/100 [==============================] - 186s 2s/step - loss: 0.1374 - acc: 0.8981 - val_loss: 0.5458 - val_acc: 0.6842
Epoch 6/10
100/100 [==============================] - 188s 2s/step - loss: 0.1397 - acc: 0.9000 - val_loss: 0.2605 - val_acc: 0.8158
Epoch 7/10
100/100 [==============================] - 190s 2s/step - loss: 0.1231 - acc: 0.9031 - val_loss: 0.6939 - val_acc: 0.7632
Epoch 8/10
100/100 [==============================] - 197s 2s/step - 

In [50]:
predictions = np.argmax(model.predict(test_image_gen), axis = -1)
predictions

49/49 [==============================] - 59s 1s/step


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 3, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3,
       0, 0, 0, 2, 3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0,
       3, 3, 0, 0, 3, 3, 3, 2, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 3, 2, 2, 3, 0, 3, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 1, 3, 2, 2, 2, 2, 3, 2, 2, 2, 0, 2, 2, 1, 2, 1, 3,
       2, 1, 0, 0, 2, 3, 3, 3, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 3,
       2, 2, 3, 2, 2, 3, 2, 3, 1, 2, 1, 1, 1, 3, 1, 3, 2, 1, 2, 3, 1, 2,
       3, 2, 2, 2, 2, 3, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 3, 2, 1, 1, 3,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 1, 2, 2, 3, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 3, 2, 1,

In [51]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_image_gen.classes,predictions))

              precision    recall  f1-score   support

           0       0.96      0.69      0.80       106
           1       0.98      0.26      0.41       234
           2       0.71      0.99      0.83       390
           3       0.46      1.00      0.63        41

    accuracy                           0.73       771
   macro avg       0.78      0.73      0.67       771
weighted avg       0.82      0.73      0.69       771



In [52]:
print(confusion_matrix(test_image_gen.classes,predictions))

[[ 73   0   5  28]
 [  3  60 151  20]
 [  0   1 388   1]
 [  0   0   0  41]]


# VGG16

In [53]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False) #

In [54]:
for layer in base_model.layers:
    layer.trainable = False
from tensorflow.keras import layers 
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)
x = layers.Dense(2048, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer with 102 node for classification output
x = layers.Dense(102, activation='relu')(x)
x = layers.Dense(4, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'BinaryCrossentropy',metrics = ['acc'])

In [55]:
vgghist = model.fit_generator(train_image_gen, validation_data = val_image_gen, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 460s 5s/step - loss: 0.3259 - acc: 0.7212 - val_loss: 0.8143 - val_acc: 0.4737
Epoch 2/10
100/100 [==============================] - 477s 5s/step - loss: 0.1914 - acc: 0.8450 - val_loss: 0.4277 - val_acc: 0.6842
Epoch 3/10
100/100 [==============================] - 453s 5s/step - loss: 0.1414 - acc: 0.8806 - val_loss: 0.1840 - val_acc: 0.8421
Epoch 4/10
100/100 [==============================] - 383s 4s/step - loss: 0.1339 - acc: 0.9019 - val_loss: 0.1785 - val_acc: 0.8421
Epoch 5/10
100/100 [==============================] - 375s 4s/step - loss: 0.1279 - acc: 0.9025 - val_loss: 0.2696 - val_acc: 0.7895
Epoch 6/10
100/100 [==============================] - 398s 4s/step - loss: 0.0962 - acc: 0.9319 - val_loss: 0.3948 - val_acc: 0.7105
Epoch 7/10
100/100 [==============================] - 399s 4s/step - loss: 0.1131 - acc: 0.9239 - val_loss: 0.4698 - val_acc: 0.7105
Epoch 8/10
100/100 [==============================] - 413s 4s/step - 

In [56]:
predictions = np.argmax(model.predict(test_image_gen), axis = -1)
predictions

49/49 [==============================] - 153s 3s/step


array([2, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3, 3, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 3,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3,
       1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [57]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_image_gen.classes,predictions))

              precision    recall  f1-score   support

           0       0.98      0.82      0.89       106
           1       0.91      0.77      0.84       234
           2       0.88      0.96      0.92       390
           3       0.69      1.00      0.82        41

    accuracy                           0.89       771
   macro avg       0.87      0.89      0.87       771
weighted avg       0.89      0.89      0.89       771



In [58]:
print(confusion_matrix(test_image_gen.classes,predictions))

[[ 87   4   2  13]
 [  1 181  47   5]
 [  1  14 375   0]
 [  0   0   0  41]]


# RESNET50

In [62]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import BatchNormalization
model = Sequential()

model.add(ResNet50(include_top=False, pooling='avg'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(4, activation='softmax'))

model.layers[0].trainable = False

In [63]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'BinaryCrossentropy',metrics = ['acc'])

In [64]:
resnet = model.fit_generator(train_image_gen, validation_data = val_image_gen, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 167s 2s/step - loss: 0.6167 - acc: 0.6506 - val_loss: 0.5748 - val_acc: 0.3158
Epoch 2/10
100/100 [==============================] - 140s 1s/step - loss: 0.4693 - acc: 0.7767 - val_loss: 0.5509 - val_acc: 0.3684
Epoch 3/10
100/100 [==============================] - 137s 1s/step - loss: 0.4068 - acc: 0.7987 - val_loss: 0.5211 - val_acc: 0.4737
Epoch 4/10
100/100 [==============================] - 143s 1s/step - loss: 0.3632 - acc: 0.8081 - val_loss: 0.5249 - val_acc: 0.4211
Epoch 5/10
100/100 [==============================] - 163s 2s/step - loss: 0.3340 - acc: 0.8194 - val_loss: 0.4971 - val_acc: 0.4737
Epoch 6/10
100/100 [==============================] - 162s 2s/step - loss: 0.3086 - acc: 0.8062 - val_loss: 0.4046 - val_acc: 0.5526
Epoch 7/10
100/100 [==============================] - 169s 2s/step - loss: 0.2724 - acc: 0.8313 - val_loss: 0.4114 - val_acc: 0.6316
Epoch 8/10
100/100 [==============================] - 167s 2s/step - 

In [65]:
predictions = np.argmax(model.predict(test_image_gen), axis = -1)
predictions

49/49 [==============================] - 76s 2s/step


array([2, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3, 2,
       0, 3, 0, 0, 3, 0, 2, 0, 3, 2, 0, 0, 0, 0, 0, 3, 0, 3, 2, 0, 2, 2,
       0, 0, 3, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 0, 0, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 2, 1, 0, 2, 0, 2, 1, 0, 0, 2, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1,
       1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2,
       1, 1, 2, 0, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2,
       1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1,
       1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1,
       2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 2, 1,

In [66]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_image_gen.classes,predictions))

              precision    recall  f1-score   support

           0       0.85      0.75      0.79       106
           1       0.90      0.55      0.68       234
           2       0.77      0.97      0.86       390
           3       0.71      0.83      0.76        41

    accuracy                           0.80       771
   macro avg       0.81      0.77      0.77       771
weighted avg       0.82      0.80      0.79       771



In [67]:
print(confusion_matrix(test_image_gen.classes,predictions))

[[ 79   1  12  14]
 [ 13 128  93   0]
 [  1  12 377   0]
 [  0   2   5  34]]
